In [1]:
# Import the function to get connect to the db
from snowexsql.db import get_db

# This is what you will use for all of hackweek to access the db
db_name = 'snow:hackweek@52.32.183.144/snowex'

# Using the function get_db, we receive 2 ways to interact with the database
engine, session = get_db(db_name)

In [2]:
# Import the class reflecting the points table in the db
from snowexsql.data import PointData

# Import the function to investigate a table
from snowexsql.db import get_table_attributes

# Use the function to see what columns are available to use. 
db_columns = get_table_attributes(PointData)

# Print out the results nicely
print("These are the available columns in the table:\n \n* {}\n".format('\n* '.join(db_columns)))

These are the available columns in the table:
 
* date
* date_accessed
* doi
* easting
* elevation
* equipment
* geom
* instrument
* latitude
* longitude
* metadata
* northing
* site_id
* site_name
* surveyors
* time
* time_created
* time_updated
* type
* units
* utm_zone
* value
* version_number



In [3]:
import snowexsql.db
from snowexsql.db import get_db
from snowexsql.data import PointData, SiteData
from snowexsql.conversions import query_to_geopandas

import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt

# Grab the open site data from the db
open_site = 'W1A'
veg_site = 'W9A'
qry = session.query(PointData).filter(PointData.equipment.contains(open_site))
df_open = query_to_geopandas(qry,engine)

# Grab the vegetated site from the db
qry = session.query(PointData).filter(PointData.equipment.contains(veg_site))
df_veg = query_to_geopandas(qry,engine)

# Set the date as the index for easy plotting/reading
df_open = df_open.set_index('date')
df_veg = df_veg.set_index('date')

In [4]:
from PIL import Image, ExifTags
import os
wd = os.getcwd()
print("Current WD: {}".format(wd))

Current WD: /home/jovyan/cam-learning/scripts


In [10]:
path_to_image = "/tmp/camera-trap/W9A/WSCT0660.JPG"


img = Image.open(path_to_image)
exif = { ExifTags.TAGS[k]: v for k, v in img._getexif().items() if k in ExifTags.TAGS }
keys = list(exif.keys())
print('\n*{}'.format('\n*'.join(keys)))


*GPSInfo
*PrintImageMatching
*ResolutionUnit
*ExifOffset
*ImageDescription
*Make
*Model
*Software
*Orientation
*DateTime
*YCbCrPositioning
*Copyright
*XResolution
*YResolution
*ExifVersion
*ComponentsConfiguration
*CompressedBitsPerPixel
*DateTimeOriginal
*DateTimeDigitized
*ShutterSpeedValue
*ApertureValue
*ExposureBiasValue
*MaxApertureValue
*MeteringMode
*LightSource
*Flash
*FocalLength
*ColorSpace
*ExifImageWidth
*SceneCaptureType
*ExifImageHeight
*Contrast
*Saturation
*Sharpness
*SensingMethod
*FileSource
*ExposureTime
*ExifInteroperabilityOffset
*FNumber
*SceneType
*ExposureProgram
*CustomRendered
*ISOSpeedRatings
*ExposureMode
*FlashPixVersion
*WhiteBalance
*DigitalZoomRatio
*CameraOwnerName
*BodySerialNumber
*FocalLengthIn35mmFilm
*LensSpecification
*LensMake
*LensModel
*LensSerialNumber
*MakerNote


Create a function to parse through all images. Save as dataframe with name of photo, date, and time. Then add SWE data by joining on dates+times. At that point we have images submit this to train.

In [11]:
from datetime import datetime
exif['DateTime'] = datetime.strptime(exif['DateTime'],'%Y:%m:%d %H:%M:%S')
print(exif['DateTime'])


2020-08-17 12:00:00


In [21]:

files = os.listdir('/tmp/camera-trap/W9A')
files =  ['/tmp/camera-trap/W9A/' + str(f) for f in files]


df = pd.DataFrame([],
                   columns=['date','photo_id','time'])
for i in range(0,len(files)): 
    
    img = Image.open(files[i])
    exif = { ExifTags.TAGS[k]: v for k, v in img._getexif().items() if k in ExifTags.TAGS }
    exif['DateTime'] = datetime.strptime(exif['DateTime'],'%Y:%m:%d %H:%M:%S')
    df.loc[i]= [exif['DateTime'].date(),
                       files[i],
                       exif['DateTime'].time()]

In [22]:
df

,date,photo_id,time
0,2019-11-19,/tmp/camera-trap/W9A/WSCT0116.JPG,12:00:00
1,2019-10-22,/tmp/camera-trap/W9A/WSCT0060.JPG,12:00:00
2,2019-10-23,/tmp/camera-trap/W9A/WSCT0061.JPG,11:00:00
3,2019-10-21,/tmp/camera-trap/W9A/WSCT0057.JPG,11:00:00
4,2019-10-19,/tmp/camera-trap/W9A/WSCT0054.JPG,12:00:01
...,...,...,...
651,2020-08-08,/tmp/camera-trap/W9A/WSCT0641.JPG,11:00:00
652,2020-08-10,/tmp/camera-trap/W9A/WSCT0645.JPG,11:00:01
653,2020-08-07,/tmp/camera-trap/W9A/WSCT0640.JPG,12:00:01
654,2020-08-14,/tmp/camera-trap/W9A/WSCT0653.JPG,11:00:00


In [1]:
session.close()

NameError: name 'session' is not defined